In [1]:
pip install yfinance backtrader ta scipy

     ---------------------------------------- 73.5/73.5 kB ? eta 0:00:00
     ------------------------------------- 419.5/419.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
     ---------------------------------------- 112.2/112.2 kB ? eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 20.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29422 sha256=5f91007aa8541229e21e81b64b72d46e13abaa68226473253e6afe272e02a1e5
  Stored in directory: c:\users\urvij\appdata

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import backtrader as bt
from scipy.optimize import brute
import matplotlib.pyplot as plt


In [3]:
def fetch_data(symbol, start, end):
    data = yf.download(symbol, start=start, end=end)
    return data

# Fetching data for TCS for a period of 4 years
data = fetch_data('TCS.NS', '2019-01-01', '2023-12-31')

data.head(10)


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-01,1896.000000,1910.000000,1885.000000,1902.800049,1687.075317,1094883
2019-01-02,1905.000000,1934.449951,1900.000000,1923.300049,1705.251465,2100463
2019-01-03,1919.000000,1944.949951,1893.099976,1899.949951,1684.548218,2611668
2019-01-04,1900.000000,1901.199951,1841.000000,1876.849976,1664.067139,4280862
2019-01-07,1891.800049,1908.800049,1881.000000,1897.900024,1682.730713,1856423
2019-01-08,1905.400024,1906.400024,1883.300049,1893.550049,1678.873901,1691756
2019-01-09,1907.400024,1919.000000,1866.699951,1886.949951,1673.022217,2414376
2019-01-10,1890.000000,1905.000000,1872.099976,1888.550049,1674.440918,3053461
2019-01-11,1870.000000,1875.000000,1835.000000,1842.550049,1633.655884,9209862


In [4]:
class SMAStrategyWithRisk(bt.Strategy):
    params = (
        ('sma1', 50),
        ('sma2', 200),
        ('stop_loss', 0.02),  # 2% stop-loss
        ('take_profit', 0.05),  # 5% take-profit
    )

    def __init__(self):
        self.sma1 = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.sma1)
        self.sma2 = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.sma2)
        self.order = None
        self.buy_price = None
        self.stop_loss_marker = []
        self.take_profit_marker = []

    def next(self):
        if self.order:
            return

        if self.sma1[0] > self.sma2[0]:
            if not self.position:
                self.buy_price = self.data.close[0]
                self.order = self.buy()
        elif self.sma1[0] < self.sma2[0]:
            if self.position:
                self.order = self.sell()

        if self.position:
            if self.position.size > 0:
                stop_price = self.buy_price * (1.0 - self.params.stop_loss)
                take_profit_price = self.buy_price * (1.0 + self.params.take_profit)
                if self.data.close[0] <= stop_price:
                    self.order = self.sell()
                    self.stop_loss_marker.append(self.data.datetime.date(0))
                elif self.data.close[0] >= take_profit_price:
                    self.order = self.sell()
                    self.take_profit_marker.append(self.data.datetime.date(0))

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price

    def plot_markers(self):
        plt.plot(self.data.close)
        for date in self.stop_loss_marker:
            plt.axvline(date, color='red', linestyle='--', lw=2, label='Stop Loss')
        for date in self.take_profit_marker:
            plt.axvline(date, color='green', linestyle='--', lw=2, label='Take Profit')
        plt.legend()


In [5]:
def run_backtest(sma1, sma2):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(SMAStrategyWithRisk, sma1=sma1, sma2=sma2)
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.broker.set_cash(1000000)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.run()
    return cerebro.broker.getvalue()

def optimize_strategy():
    def objective(params):
        sma1, sma2 = params
        return -run_backtest(sma1, sma2)  # Negative for minimization

    ranges = (slice(10, 100, 10), slice(100, 300, 10))
    result = brute(objective, ranges, finish=None)
    return result

optimal_params = optimize_strategy()
print(f"Optimal SMA1: {optimal_params[0]}, Optimal SMA2: {optimal_params[1]}")


Optimal SMA1: 30.0, Optimal SMA2: 190.0


In [6]:
cerebro = bt.Cerebro()
cerebro.addstrategy(SMAStrategyWithRisk, sma1=int(optimal_params[0]), sma2=int(optimal_params[1]))
data_bt = bt.feeds.PandasData(dataname=data)
cerebro.adddata(data_bt)
cerebro.broker.set_cash(1000000)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
results = cerebro.run()

# Get the strategy instance
strategy = results[0]

# Plot the backtest result with candlestick style
cerebro.plot(style='candlestick')

# Plot stop-loss and take-profit markers
plt.figure(figsize=(10, 5))
plt.plot(data['Close'], label='Close Price')
strategy.plot_markers()
plt.title('Stop Loss and Take Profit Markers')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>